In [ ]:
!pip install deslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.6/172.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from deslib.dcs import OLA
from deslib.dcs import LCA
from deslib.dcs import MCB
from deslib.des import KNORAE
from deslib.des import KNORAU
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier

url = "https://raw.githubusercontent.com/tmoura/machinelearning/master/datasets/ionosphere.data"

# Carregar base de dados
# DataFrame
dataset = pd.read_csv(url, header=None)

columns = len(dataset.columns)

y = dataset[0] # extrai a primeira coluna, que é o label
X = dataset.loc[:,1:columns-1]

# Transforma para Array NumPy
X = np.array(X)
y = np.array(y)

In [ ]:
# 1) Usar o dataset gerado na célula acima para criar 10 folds estratificados de Treinamento e Teste
from sklearn.model_selection import StratifiedKFold

#Quantidade de folds
folds = 10

kf = StratifiedKFold(n_splits = folds)

#Listas para as estratificações de treinamento e de teste
X_train = []
y_train = []

X_test = []
y_test = []

#Gerando as estratificações
for train_index, test_index in kf.split(X,y):
  X_train.append(X[train_index])
  X_test.append(X[test_index])

  y_train.append(y[train_index])
  y_test.append(y[test_index])

In [ ]:
# 2) Treinar 10 Árvores de Decisão e 10 RandomForest (com 50 árvores) utilizando os conjuntos de Treinamentos gerados na célula anterior
# 3) Calcular os 10 resultados das acurácias nos 10 conjuntos de Teste
# 4) Printar o resultado da média das 10 acurácias para as Árvores e os RandomForest

tree_results = []
randomforest_results = []

for i in range(folds):
  #Instancia dos modelos
  tree = DecisionTreeClassifier(criterion="entropy", max_depth=8)
  randomforest = RandomForestClassifier(n_estimators=50)

  #Treinamento dos modelos
  tree.fit(X_train[i], y_train[i])
  randomforest.fit(X_train[i], y_train[i])

  #Acurácia dos modelos
  tree_results.append(tree.score(X_test[i],y_test[i]))
  randomforest_results.append(randomforest.score(X_test[i], y_test[i]))

print("RESULTADOS: ")
print(f"Média das acurácias das Árvores de Decisão {np.mean(tree_results)}")
print(f"Média das acurácias das Florestas Aleatórias {np.mean(randomforest_results)}")

RESULTADOS: 
Média das acurácias das Árvores de Decisão 0.8775396825396825
Média das acurácias das Florestas Aleatórias 0.9087301587301587


In [ ]:
# 5) Treinar:
  # 10 Modelos Bagging (com 50 árvores) - O Bagging do SkLearn já utiliza Árvores de Decisão como modelo padrão do Ensemble
  # 10 KNORA-U
  # 10 KNORA-E
  # 10 LCA
  # 10 OLA
  # 10 MCB

  # Usar nos modelos de seleção dinâmica as árvores geradas pelo Bagging

# 6) Calcular os 10 resultados das acurácias nos 10 conjuntos de Teste para todos os modelos
# 7) Printar o resultado da média das 10 acurácias para todos os modelos

bagging_results = []
knorae_results = []
knorau_results = []
ola_results = []
lca_results = []
mcb_results = []

for i in range(folds):
  #Instancia do Bagging, treino e resultados com 50 arvores
  bagging = BaggingClassifier(n_estimators = 50)
  bagging.fit(X_train[i], y_train[i])
  bagging_results.append(bagging.score(X_test[i], y_test[i]))

  #Separação dos dados para treinos
  train_X, X_dsel, train_y , y_dsel = train_test_split(X_train[i], y_train[i], test_size = 0.5, random_state = None)


  knorae = KNORAE(bagging, k=5)
  knorau = KNORAU(bagging, k=5)
  ola = OLA(bagging, k=5)
  lca = LCA(bagging, k=5)
  mcb = MCB(bagging, k=5)

  knorae.fit(X_dsel, y_dsel)
  knorau.fit(X_dsel, y_dsel)
  ola.fit(X_dsel, y_dsel)
  lca.fit(X_dsel, y_dsel)
  mcb.fit(X_dsel, y_dsel)

  knorae_results.append(knorae.score(X_test[i], y_test[i]))
  knorau_results.append(knorau.score(X_test[i], y_test[i]))
  ola_results.append(ola.score(X_test[i], y_test[i]))
  lca_results.append(lca.score(X_test[i], y_test[i]))
  mcb_results.append(mcb.score(X_test[i], y_test[i]))

print("RESULTADOS:")
print(f"Média das acurácias dos Bagging's: {np.mean(bagging_results)}")
print(f"Média das acurácias dos KNORA-E's: {np.mean(knorae_results)}")
print(f"Média das acurácias dos KNORA-U's: {np.mean(knorau_results)}")
print(f"Médias das acurácias dos OLA's: {np.mean(ola_results)}")
print(f"Médias das acurácias dos LCA's: {np.mean(lca_results)}")
print(f"Médias das acurácias dos MCB's: {np.mean(mcb_results)}")

RESULTADOS:
Média das acurácias dos Bagging's: 0.8974603174603175
Média das acurácias dos KNORA-E's: 0.8974603174603175
Média das acurácias dos KNORA-U's: 0.8974603174603175
Médias das acurácias dos OLA's: 0.9003174603174603
Médias das acurácias dos LCA's: 0.9173809523809524
Médias das acurácias dos MCB's: 0.8773015873015874
